In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

from s4hci.utils.data_handling import load_adi_data, save_as_fits

# Load the dataset

In [2]:
dataset_hdf5_file = "/fast/mbonse/s4/30_data/HD22049_304_199_C-0065_C_.hdf5"

In [3]:
science_data, raw_angles, raw_psf_template_data = \
    load_adi_data(
        hdf5_dataset=dataset_hdf5_file,
        data_tag="object",
        psf_template_tag="psf_template",
        para_tag="header_object/PARANG")

science_data = science_data[:, 12:-12, 12:-12]

## stack the frames

In [4]:
num_frames = science_data.shape[0]
stacking_n_frames = 10

In [5]:
stacking_idx = np.array_split(np.arange(num_frames), 
                              int(num_frames / stacking_n_frames))

In [6]:
stacked_science = np.array([np.mean(science_data[tmp_idx], axis=0) for tmp_idx in stacking_idx])
stacked_angles = np.array([np.median(raw_angles[tmp_idx], axis=0) for tmp_idx in stacking_idx])

In [7]:
stacked_science.shape

(5756, 117, 117)

In [8]:
stacked_angles.shape

(5756,)

# Run a CPU based cADI

In [9]:
from s4hci.utils.adi_tools import combine_residual_stack

In [10]:
median_frame = np.median(stacked_science, axis=0)

In [11]:
residual_sequence = stacked_science - median_frame

In [12]:
residual_image = combine_residual_stack(
    residual_stack=residual_sequence,
    angles=stacked_angles,
    combine="mean")